In [17]:
import pandas as pd

# Đọc dữ liệu thô
# Chúng ta cần chỉ định tên cột vì file CSV thô có thể không có header
df_raw = pd.read_csv('../data/raw/Phishing_Email.csv', header=None, names=['Email Text', 'Email Type'])

print("--- Dữ liệu thô ban đầu ---")
print(df_raw.head())
print(f"\nSố lượng email ban đầu: {len(df_raw)}")

--- Dữ liệu thô ban đầu ---
                                            Email Text      Email Type
NaN                                         Email Text      Email Type
0.0  re : 6 . 1100 , disc : uniformitarianism , re ...      Safe Email
1.0  the other side of * galicismos * * galicismo *...      Safe Email
2.0  re : equistar deal tickets are you still avail...      Safe Email
3.0  \nHello I am your hot lil horny toy.\n    I am...  Phishing Email

Số lượng email ban đầu: 18651


In [19]:
# --- Xử lý giá trị thiếu (dropna) ---
print(f"Số dòng bị thiếu dữ liệu trước khi xử lý: {df_raw.isnull().sum().sum()}")
df_cleaned = df_raw.dropna().copy() # Dùng .copy() để tránh warning
print("Đã xóa các dòng bị thiếu dữ liệu.")

# --- Loại bỏ dữ liệu trùng lặp (drop_duplicates) ---
print(f"\nSố dòng bị trùng lặp trước khi xử lý: {df_cleaned.duplicated().sum()}")
df_cleaned.drop_duplicates(inplace=True)
print("Đã xóa các dòng bị trùng lặp.")
print(f"Số dòng còn lại: {len(df_cleaned)}")

Số dòng bị thiếu dữ liệu trước khi xử lý: 16
Đã xóa các dòng bị thiếu dữ liệu.

Số dòng bị trùng lặp trước khi xử lý: 1096
Đã xóa các dòng bị trùng lặp.
Số dòng còn lại: 17539


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
import re # Thư viện cho biểu thức chính quy (regular expressions)
import string # Thư viện chứa các dấu câu
# Có thể bạn cần tải stop words lần đầu
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

# Lấy danh sách các từ dừng tiếng Anh
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # --- Chuyển thành chữ thường ---
    text = text.lower()
    
    # --- Loại bỏ URLs ---
    # Mẫu này sẽ tìm tất cả các link http, https, hoặc www
    text = re.sub(r'https|http?://\S+|www\.\S+', '', text)
    
    # --- Loại bỏ dấu câu ---
    # string.punctuation chứa tất cả các dấu câu như '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    
    # # --- Loại bỏ số ---
    # text = re.sub(r'\d+', '', text)
    
    # --- (Tùy chọn) Loại bỏ stop words ---
    # Tách câu thành các từ, sau đó chỉ giữ lại những từ không phải là stop word
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    text = ' '.join(filtered_words)
    
    # --- Loại bỏ khoảng trắng thừa ---
    text = ' '.join(text.split())
    
    return text

print("Hàm preprocess_text đã sẵn sàng!")

Hàm preprocess_text đã sẵn sàng!


In [22]:
# Áp dụng hàm preprocess_text cho toàn bộ cột 'Email Text'
# Quá trình này có thể mất một lúc tùy vào kích thước file
df_cleaned['clean_text'] = df_cleaned['Email Text'].apply(preprocess_text)

print("--- SO SÁNH TRƯỚC VÀ SAU KHI LÀM SẠCH ---")

# Lấy ra một email thô để làm ví dụ
raw_example = df_cleaned['Email Text'].iloc[11] 
# iloc[11] là email có URL: "URL: http://www.newsis...."

# Lấy ra email tương ứng đã được làm sạch
cleaned_example = df_cleaned['clean_text'].iloc[11]

print("\nEMAIL GỐC:\n", raw_example)
print("\nEMAIL ĐÃ LÀM SẠCH:\n", cleaned_example)

--- SO SÁNH TRƯỚC VÀ SAU KHI LÀM SẠCH ---

EMAIL GỐC:
 make her beg you to give it to her everynight you too can please you and your partner so much better . and it doesn ' t take half a year like other solutions . if you don ' t trust me , trust the new york times , newsweek , time magazine , and the testemonials of many people all over the net . look , if anything else check out the site and see the facts for yourself its new , its safe ! its the most advanced penile enlargement solution ! it ' s 100 % guaranteed to enlarge your penis . 3 + inches magna - rx patch - click here the amazing , new magna rx patch is not available in any stores or on other websites . accept no imitations ! order your male enhancement patch now through this exclusive website offer and get a 1 - month supply free ! one small investment in yourself will last a lifetime ! - no pills or capsules - no lotions or cremes - no pumps , weights , or exercises - no prescription necessary - doctor designed endorsed - 

In [27]:
# Chuyển đổi cột 'Email Type' (dạng chữ) thành cột 'label' (dạng số 0 và 1)
# 0 cho 'Safe Email', 1 cho 'Phishing Email'
df_cleaned['label'] = df_cleaned['Email Type'].map({'Safe Email': 0, 'Phishing Email': 1})

# Tạo DataFrame cuối cùng chỉ với 2 cột chúng ta cần
final_df = df_cleaned[['clean_text', 'label']].copy()

# Đổi tên cột 'clean_text' thành 'text_combined' để giống hệt file mẫu
final_df.rename(columns={'clean_text': 'text_combined'}, inplace=True)

# --- CHỈNH SỬA Ở ĐÂY ---
# Chuyển đổi kiểu dữ liệu của cột 'label' từ float (1.0) sang integer (1)
# Và loại bỏ các dòng có label là NaN (phát sinh nếu có giá trị 'Email Type' lạ)
final_df.dropna(subset=['label'], inplace=True)
final_df['label'] = final_df['label'].astype(int)
# -------------------------

print("--- DATAFRAME CUỐI CÙNG ĐÃ CHUẨN HÓA ---")
print(final_df.head())

print("\nKiểm tra lại thông tin và kiểu dữ liệu:")
print(final_df.info())


# (Tùy chọn) Lưu DataFrame sạch này ra một file CSV mới
# Đảm bảo thư mục 'clear' đã tồn tại trong thư mục 'data'
final_df.to_csv('../data/clear/my_cleaned_phishing_data.csv', index=False)

--- DATAFRAME CUỐI CÙNG ĐÃ CHUẨN HÓA ---
                                         text_combined  label
0.0  disc uniformitarianism sex lang dick hudson ob...      0
1.0  side galicismos galicismo spanish term names i...      0
2.0  equistar deal tickets still available assist r...      0
3.0  hello hot lil horny toy one dream open minded ...      1
4.0  software incredibly low prices lower drapery s...      1

Kiểm tra lại thông tin và kiểu dữ liệu:
<class 'pandas.core.frame.DataFrame'>
Index: 17538 entries, 0.0 to 18649.0
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_combined  17538 non-null  object
 1   label          17538 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 411.0+ KB
None
